<a href="https://colab.research.google.com/github/tomonari-masada/course2025-stats2/blob/main/07_linear_regression_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 階層的ベイズ線形回帰

* 参考資料
  * https://medium.com/analytics-vidhya/higher-spending-leads-to-poorer-education-a-bayesian-statistics-project-using-rjags-b50b213c6961

## 準備

In [ ]:
!pip install numpyro

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import jax
import jax.numpy as jnp
from jax import random
import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro.infer import NUTS, MCMC, Predictive

import arviz as az

%config InlineBackend.figure_format = 'retina'

plt.rcParams.update({'figure.constrained_layout.use':True})
plt.style.use("bmh")
az.style.use("arviz-darkgrid")

rng_key = random.PRNGKey(0)

numpyro.set_platform("cpu")

## ベイズ線形回帰

### ベイズ推論
* やはり、誤差項が正規分布に従うと仮定する。
$$ \epsilon \sim N(0, \sigma^2)$$
* これを書き直すと
$$ Y \sim N(\beta^\top X, \sigma^2) $$
* ベイズ推論を使う場合、$\beta$や$\sigma$が従う分布を、事前分布として導入する。
* そして、$\beta$や$\sigma$が従う事後分布を求める。

## 階層的線形モデル
* https://www.pymc.io/projects/docs/en/v3/pymc-examples/examples/generalized_linear_models/GLM.html


* 重回帰分析を行う。
* 係数がどのような分布に従うかの仮定を変えて、分析してみる。
  * 正規分布に従うと仮定したモデル。
  * t分布に従うと仮定したモデル。
* 係数が従う分布のlocationとscaleも、それぞれ特定の分布に従うと仮定する。→**階層モデル**
  * 平均パラメータは、正規分布に従うと仮定する。
  * 精度パラメータは、ガンマ分布に従うと仮定する。

### データ
* データの説明
  * 州の名前
  * `spend`: Current expenditure per pupil in average daily attendance in public elementary and secondary schools, 1994-95 (in thousands of dollars)
  * `stu_tea_rat`: Average pupil/teacher ratio in public elementary and secondary schools, Fall 1994
  * `salary`: Estimated average annual salary of teachers in public elementary and secondary schools, 1994-95 (in thousands of dollars)
  * `prcnt_take`: Percentage of all eligible students taking the SAT, 1994-95
  * `sat_v`: Average verbal SAT score, 1994-95
  * `sat_m`: Average math SAT score, 1994-95
  * `sat_t`:	Average total score on the SAT, 1994-95
* 今回のモデリングにおける目的変数は`sat_t`とする。

In [ ]:
!wget https://raw.githubusercontent.com/pymc-devs/pymc-examples/main/examples/data/Guber1999data.txt
sat_data = pd.read_csv("Guber1999data.txt")

In [ ]:
sat_data.head()

In [ ]:
sat_data.describe()

In [ ]:
sat_data.info()

### EDA

* 以下、`sat_v`と`sat_m`は除外する。
  * 今回はSATの合計点だけに関心があるので。

In [ ]:
sat_data = sat_data.drop(columns=["sat_v", "sat_m"])

In [ ]:
sns.pairplot(sat_data);

* `spend`と`sat_t`の相関を見てみる。

In [ ]:
g = sns.relplot(sat_data, x='spend', y='sat_t', hue='prcnt_take')
g._legend._loc = 1

* 一見すると、州がお金を掛けるほど、SATの合計スコアが落ちているように見える。（全体的に右下がりなので。）
  * これは直感に反するが・・・
* しかし、お金を掛けるほど、受験率も上がっている。（右のほうが色が濃くなっているので。）
* ということは、受験率が上がることで、成績が下位の学生も受験することになり、そのため、スコアが落ちているのではないか。
* よって、お金を掛けるほどスコアが落ちる、というわけではなさそう。

* NumPyroで使いやすいようにデータフレームを辞書に作りかえておく。
  * 各カラムの数値の列はJAXの配列に変換する。

In [ ]:
kwargs = dict((col, jnp.array(sat_data[col].values)) for col in sat_data.columns)
kwargs

### モデル1
* 係数が正規分布に従うと仮定する。
* この正規分布のパラメータは、定数とする。
  * つまり、階層的なモデリングはおこなわない。

In [ ]:
def model1(spend=None, stu_tea_rat=None, salary=None, prcnt_take=None, sat_t=None):
  grp_mean = 0
  grp_sd = 20

  intercept = numpyro.sample("intercept", dist.Normal(sat_data.sat_t.mean(), sat_data.sat_t.std()))

  coef_spend = numpyro.sample("coef_spend", dist.Normal(grp_mean, grp_sd))
  coef_stu_tea_rat = numpyro.sample("coef_stu_tea_rat", dist.Normal(grp_mean, grp_sd))
  coef_salary = numpyro.sample("coef_salary", dist.Normal(grp_mean, grp_sd))
  coef_prcnt_take = numpyro.sample("coef_prcnt_take", dist.Normal(grp_mean, grp_sd))

  sd = numpyro.sample("sd", dist.HalfCauchy(10))

  mu = (
      intercept
      + coef_spend * spend
      + coef_stu_tea_rat * stu_tea_rat
      + coef_salary * salary
      + coef_prcnt_take * prcnt_take
      )
  numpyro.sample("obs", dist.Normal(mu, sd), obs=sat_t)

In [ ]:
rng_key, rng_key_ = random.split(rng_key)
kernel = NUTS(model1)
mcmc = MCMC(kernel, num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(rng_key_, **kwargs)

In [ ]:
mcmc.print_summary()

* `coef_spend`の`std`がかなり大きいことに注意。
  * この点が、[参考資料](https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fmedium.com%2Fanalytics-vidhya%2Fhigher-spending-leads-to-poorer-education-a-bayesian-statistics-project-using-rjags-b50b213c6961)でも議論されている。


In [ ]:
(mcmc.get_samples()['coef_spend'] > 0.0).sum() / (mcmc.num_samples * mcmc.num_chains)

In [ ]:
idata1 = az.from_numpyro(mcmc)
az.plot_trace(idata1);

### モデル2
* 係数が正規分布に従うと仮定する。
* さらに、この正規分布のパラメータに事前分布を導入する。
  * つまり、係数について階層的なモデリングをおこなう。

In [ ]:
def model2(spend=None, stu_tea_rat=None, salary=None, prcnt_take=None, sat_t=None):
  grp_mean = numpyro.sample("grp_mean", dist.Normal(0, 10))
  grp_sd = numpyro.sample("grp_sd", dist.HalfCauchy(5))

  intercept = numpyro.sample("intercept", dist.Normal(sat_data.sat_t.mean(), sat_data.sat_t.std()))

  coef_spend = numpyro.sample("coef_spend", dist.Normal(grp_mean, grp_sd))
  coef_stu_tea_rat = numpyro.sample("coef_stu_tea_rat", dist.Normal(grp_mean, grp_sd))
  coef_salary = numpyro.sample("coef_salary", dist.Normal(grp_mean, grp_sd))
  coef_prcnt_take = numpyro.sample("coef_prcnt_take", dist.Normal(grp_mean, grp_sd))

  sd = numpyro.sample("sd", dist.HalfCauchy(10))

  mu = (
      intercept
      + coef_spend * spend
      + coef_stu_tea_rat * stu_tea_rat
      + coef_salary * salary
      + coef_prcnt_take * prcnt_take
      )
  numpyro.sample("obs", dist.Normal(mu, sd), obs=sat_t)

In [ ]:
rng_key, rng_key_ = random.split(rng_key)
kernel = NUTS(model2)
mcmc = MCMC(kernel, num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(rng_key_, **kwargs)

In [ ]:
mcmc.print_summary()

* `coef_spent`の標準偏差が先ほどより小さくなっている。
  * 階層的なモデリングでなかったために、大きな値となっていた可能性もある。

In [ ]:
(mcmc.get_samples()['coef_spend'] > 0.0).sum() / (mcmc.num_samples * mcmc.num_chains)

In [ ]:
idata2 = az.from_numpyro(mcmc)
az.plot_trace(idata2);

In [ ]:
models = { "model 1": idata1, "model 2": idata2 }
df_compare = az.compare(models)
df_compare

In [ ]:
az.plot_compare(df_compare);

* モデル2を採用する方が良さそう。

### モデル3
* 誤差項がt分布に従うと仮定する。
  * そしてこのt分布のパラメータにも事前分布を導入する。

In [ ]:
def FoldedStudentT(df, loc=0.0, scale=1.0):
  return dist.FoldedDistribution(dist.StudentT(df, loc=loc, scale=scale))

In [ ]:
def model3(spend=None, stu_tea_rat=None, salary=None, prcnt_take=None, sat_t=None):
  grp_mean = numpyro.sample("grp_mean", dist.Normal(0, 10))
  grp_sd = numpyro.sample("grp_sd", dist.HalfCauchy(5))

  intercept = numpyro.sample("intercept", dist.Normal(sat_data.sat_t.mean(), sat_data.sat_t.std()))

  coef_spend = numpyro.sample("coef_spend", dist.Normal(grp_mean, grp_sd))
  coef_stu_tea_rat = numpyro.sample("coef_stu_tea_rat", dist.Normal(grp_mean, grp_sd))
  coef_salary = numpyro.sample("coef_salary", dist.Normal(grp_mean, grp_sd))
  coef_prcnt_take = numpyro.sample("coef_prcnt_take", dist.Normal(grp_mean, grp_sd))

  sigma0 = numpyro.sample("sigma0", dist.HalfCauchy(10))
  nu0 = numpyro.sample("nu0", dist.Gamma(2, 0.1))
  sd = numpyro.sample("sigma", FoldedStudentT(nu0, scale=sigma0))
  nu = numpyro.sample("nu", dist.Gamma(2, 0.1))

  mu = (
      intercept
      + coef_spend * spend
      + coef_stu_tea_rat * stu_tea_rat
      + coef_salary * salary
      + coef_prcnt_take * prcnt_take
      )
  numpyro.sample("obs", dist.StudentT(nu, mu, sd), obs=sat_t)

In [ ]:
rng_key, rng_key_ = random.split(rng_key)
kernel = NUTS(model2)
mcmc = MCMC(kernel, num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(rng_key_, **kwargs)

In [ ]:
mcmc.print_summary()

* モデル2と大きな違いはなさそう。

In [ ]:
(mcmc.get_samples()['coef_spend'] > 0.0).sum() / (mcmc.num_samples * mcmc.num_chains)

In [ ]:
idata3 = az.from_numpyro(mcmc)
az.plot_trace(idata3);

In [ ]:
models = { "model 2": idata2, "model 3": idata3 }
df_compare = az.compare(models)
df_compare

In [ ]:
az.plot_compare(df_compare);

* モデル2とモデル3の違いはなさそう。
  * 目的変数の値に外れ値が含まれる可能性を考える必要はなさそう。

* 係数の事後分布のpair plotを描いてみる。

In [ ]:
az.plot_pair(
    idata3,
    kind=["scatter", "kde"],
    marginals=True,
    point_estimate="median",
    figsize=(15, 12),
    textsize=12
);

* 目的変数の値にモデルがうまくフィットしている状態で・・・
* `spend`の係数をかなり増やしても、`salary`の係数を相応に減らせば、フィットした状態は維持できる。
* 二つの係数をどのように決めればいいかには任意性がある、ということ。